In [0]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 1s (663 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 134448 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 784 kB of 

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [0]:
!pip install ptan
import torch
import torch.nn as n
import torch.nn.functional as f
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as op
import gym
import ptan

     |████████████████████████████████| 773.1MB 22kB/s 
  Created wheel for ptan: filename=ptan-0.6-cp36-none-any.whl size=23502 sha256=3793fd8e56baec51aa2d82f2b7c9589b0e1ec94b2298b0ee4ecf838cd79bbaed
  Stored in directory: /root/.cache/pip/wheels/f0/4b/2f/9a45fd39b0a614a2716bc6128a7f1adb4647f323a2d90783f2
Successfully built ptan
ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0


In [0]:
env=gym.make("CartPole-v0")
nactions=env.action_space.n
obssize=env.observation_space.shape[0]

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
class net(n.Module):
  def __init__(self,indim,middim,nactions):
    super().__init__()
    self.fc1=n.Linear(indim,middim)
    self.fc2=n.Linear(middim,nactions)
  
  def forward(self,x):
    out=f.relu(self.fc1(x))
    out=self.fc2(out)
    return out


In [0]:
gamma=0.9
model1=net(obssize,128,nactions)

In [0]:
def calc_qvals(reward):
  batchreward=0
  qvals=[]
  for i in reversed(reward):
    batchreward*=gamma
    batchreward+=i
    qvals.append(batchreward)
  return list(reversed(qvals))

In [0]:
model1=net(4,128,2)

In [0]:
calc_qvals([1,2,3,4])

[8.146, 7.9399999999999995, 6.6, 4.0]

In [0]:
agent=ptan.agent.PolicyAgent(model1,preprocessor=ptan.agent.float32_preprocessor,apply_softmax=True)

In [0]:
expsource=ptan.experience.ExperienceSourceFirstLast(env,agent,gamma=0.9,steps_count=1)

In [0]:
#expsource.pop_total_rewards() gives sum of all rewards for that episode till terminal state is reached

In [0]:
optimizer=op.Adam(model1.parameters(),lr=0.001)

In [0]:
totalrewards=[]
stepidx=0
doneepisodes=0
batchepisodes=0
batchstates,batchactions,batchqval=[],[],[]
curreward=[]
episodestotrain=64 # sort of like batch but not done at once done in stochastic way

In [0]:
count=0
for stepidx,exp in enumerate(expsource):
  batchstates.append(exp.state)
  batchactions.append(int(exp.action))
  curreward.append(exp.reward)

  if exp.last_state is None:
    batchqval.extend(calc_qvals(curreward))
    curreward.clear()
    batchepisodes+=1

  newrewards=expsource.pop_total_rewards()
  if newrewards:
    doneepisodes+=1
    reward=newrewards[0]
    totalrewards.append(reward)
    mean_rewards=float(np.mean(totalrewards[-100:]))
  
    count+=1
  if count%10:
    print(reward," ",mean_rewards)

    if mean_rewards>195:
      print("Solved")
      break

  if batchepisodes<episodestotrain:
    continue

  optimizer.zero_grad()
  statesv=torch.FloatTensor(batchstates)
  actionv=torch.LongTensor(batchactions)
  batchqvalv=torch.FloatTensor(batchqval)

  logits=model1.forward(statesv)
  logprobv=f.log_softmax(logits,dim=1)

  print(batchqvalv.shape)
  print(logprobv.shape)
  print(actionv)
  print(len(batchstates))
  logprobaction=batchqvalv*logprobv[range(len(batchstates)),actionv]
  lossv=-logprobaction.mean()

  break
  lossv.backward()
  optimizer.step()
  batchepisodes=0
  batchstates.clear()
  batchactions.clear()
  batchqval.clear()
  t+=1
  if t==5:
    break

torch.Size([1455])
torch.Size([1456, 2])
tensor([1, 1, 0,  ..., 1, 0, 1])
1456


RuntimeError: ignored

In [0]:
a=torch.randint(0,10,(10,4))

In [0]:
b=torch.randint(0,4,(10,))

In [0]:
b

tensor([2, 2, 3, 2, 0, 3, 3, 0, 2, 1])

In [0]:
a[range(10),b]

tensor([6, 3, 6, 3, 5, 6, 1, 3, 8, 0])

In [0]:
a

tensor([[2, 4, 6, 7],
        [9, 8, 3, 0],
        [2, 2, 5, 6],
        [5, 4, 3, 6],
        [5, 6, 7, 5],
        [4, 4, 0, 6],
        [5, 0, 2, 1],
        [3, 0, 4, 2],
        [1, 1, 8, 3],
        [9, 0, 6, 6]])

In [0]:
#very important understanding
#actionv are index of actions  in 0,1,2,3,4,5
# probaction are normalized probabilites of shape batch x nactions normalized along dimension 1